In [1]:
### Import the required libraries
import numpy as np
import matplotlib.pyplot as plt 

import warnings
warnings.simplefilter("ignore")

import tensorflow as tf
import tensorflow.keras as keras
tf.compat.v1.disable_eager_execution()

import os
from os.path import join
import sys

import xarray as xr
import xmitgcm
from xmitgcm import open_mdsdataset
import ecco_v4_py as ecco

# Append to sys.path the absolute path to src/XAIRT
path_list = os.path.abspath('').split('/')
path_src_XAIRT = ''
for link in path_list[:-1]:
    path_src_XAIRT = path_src_XAIRT+link+'/'
sys.path.append(path_src_XAIRT+'/src')

# Now import module XAIRT
from XAIRT import *

In [2]:
mainDir  = '/scratch2/pillarh/eccov4r4'
gridDir  = mainDir + '/GRID'
thetaDir = mainDir + '/eccov4r4_nctiles_daily/THETA'

In [ ]:
def subset(ds):
    dict_subset = {'k':0}
    ds_subset = ds.isel(dict_subset)
    return ds_subset

ds = xr.open_mfdataset(join(thetaDir,'*/*/*.nc'), preprocess=subset); # ; at the end to suppress output
ds.to_netcdf('/scratch2/shreyas/thetaSurfECCOv4r4.nc', mode='w')

In [ ]:
ds = xr.open_dataset('/scratch2/shreyas/thetaSurfECCOv4r4.nc')

In [ ]:
hFacC = ecco.read_llc_to_tiles(gridDir, 'hFacC.data')
hFacC_mask = hFacC > 0
hFacC_mask = hFacC_mask.astype(float)

XC = ds['XC'].data
YC = ds['YC'].data

latMask = YC > -20.0
latMask = latMask.astype(float)

maskFinal = hFacC_mask * latMask
NaNmaskFinal = np.copy(maskFinal)
NaNmaskFinal[NaNmaskFinal == 0] = np.nan

da_hFacC_mask = xr.DataArray(
    data=hFacC_mask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds['tile'].data,
        j    = ds['j'].data,
        i    = ds['i'].data,
    ),
    attrs=dict(description="hFacC mask 3D 1 if > 0, else 0"),
)

da_latMask = xr.DataArray(
    data=latMask,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds['tile'].data,
        j    = ds['j'].data,
        i    = ds['i'].data,
    ),
    attrs=dict(description="Latitude Mask 1 if > -20, else 0"),
)

da_maskFinal = xr.DataArray(
    data=maskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds['tile'].data,
        j    = ds['j'].data,
        i    = ds['i'].data,
    ),
    attrs=dict(description="hFacC mask 2D 1 if > 0, else 0"),
)

da_NaNmaskFinal = xr.DataArray(
    data=NaNmaskFinal,
    dims=["tile", "j", "i"],
    coords=dict(
        tile = ds['tile'].data,
        j    = ds['j'].data,
        i    = ds['i'].data,
    ),
    attrs=dict(description="hFacC mask 2D True if > 0, else NaN"),
)

ds = ds.assign(hFacC_mask   = da_hFacC_mask,
          latMask      = da_latMask,
          maskFinal    = da_maskFinal,
          NaNmaskFinal = da_NaNmaskFinal)

wetpoints = np.nonzero(ds['maskFinal'].data)

In [ ]:
plt.pcolor(ds['XC'].isel(tile = 10), ds['YC'].isel(tile = 10), 
           ds['NaNmaskFinal'].isel(tile = 10)*ds['THETA'].isel(time = 0, tile = 10), 
           cmap = 'jet', vmax = 35)
plt.scatter(-126.0, 50.0, color = 'brown', label = '234E, 50 N')
plt.scatter(x.isel(j=1,i=43), y.isel(j=1,i=43), color = 'black', label = 'Objective function point')
plt.colorbar()
plt.legend()

In [ ]:
plt.figure(figsize=(15,5))

P = ecco.plot_proj_to_latlon_grid(ds['XC'], ds['YC'],
                              ds['THETA'].isel(time = 0),
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmax = 35,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [1,2,1]);
P[1].set_title("Theta 1992_01_01")

P = ecco.plot_proj_to_latlon_grid(ds['XC'], ds['YC'],
                              ds['NaNmaskFinal']*ds['THETA'].isel(time = 0),
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmax = 35,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [1,2,2]);
P[1].set_title("MaskedTheta 1992_01_01")

In [ ]:
X_2d = ds['THETA'].transpose("time","j","i","tile").data
### X_2d is transposed, so order of wetpoint array changes, note in next line
X    = X_2d[:,wetpoints[1],wetpoints[2],wetpoints[0]]
y    = ds['THETA'].isel(tile = 10, j = 1, i = 43).data

In [ ]:
def quickSetup(X, y, numVars, lrp_methods, lagSteps, **NNkwargs):
    
    result = {}
    
    if lagSteps > 0:
        K = TrainFullyConnectedNN(X[:-lagSteps], y[lagSteps:], **NNkwargs)
        L = TrainLR(X[:-lagSteps], y[lagSteps:], y_ref = 0.0, fit_intercept = False)
    if lagSteps == 0:
        K = TrainFullyConnectedNN(X, y, **NNkwargs)
        L = TrainLR(X, y, y_ref = 0.0, fit_intercept = False)
        
    best_model = K.quickTrain()
    result['cost_predict_NN'] = best_model.predict(X)
    
    regr = L.quickTrain()
    result['cost_predict_LR'] = regr.predict(X)
    
    normalizeDict = {'bool_': True, 'kind': 'MaxAbs'}
    kwargs = {'y_ref': 0.00}

    for method in lrp_methods:
        
        print(f'Analyze using {method}')
        
        Xplain = XAIR(best_model, method, 'classic', X, normalizeDict, **kwargs)
        a, _  = Xplain.quick_analyze()
        perVar = int(a.shape[1]/numVars)
        
        rel = np.zeros((numVars, a.shape[0], 13, 90, 90))
        rel[:,:,:,:,:] = np.nan
        
        for numvar in range(numVars):
            rel[numvar,:,wetpoints[0],wetpoints[1],wetpoints[2]] = a[:,numvar*perVar:(numvar+1)*perVar].T
        result[method] = rel
        
    #     print(f'Analyze using lrp.LR')

    #     XL = XLR(regr, X)
    #     a_LR, _ = XL.quick_analyze()
    #     perVar = int(a_LR.shape[1]/numVars)

    #     rel_LR = np.zeros((numVars, a_LR.shape[0], 13, 90, 90))
    #     rel_LR[:,:,:,:,:] = np.nan

    #     for numvar in range(numVars):
    #         rel_LR[numvar,:,wetpoints[0],wetpoints[1],wetpoints[2]] = a_LR[:,numvar*perVar:(numvar+1)*perVar].T
    #     result['lrp.LR'] = rel_LR
        
    return result

In [ ]:
Layers = [{'size': X.shape[1]    , 'activation': None    , 'use_bias': None},
          {'size': 100           , 'activation': 'relu'  , 'use_bias': True},
          {'size': 80            , 'activation': 'relu'  , 'use_bias': True},
          {'size': 60            , 'activation': 'relu'  , 'use_bias': True},
          {'size': 40            , 'activation': 'relu'  , 'use_bias': True},
          {'size': 20            , 'activation': 'relu'  , 'use_bias': True},
          {'size': 1             , 'activation': 'linear', 'use_bias': False}]
Losses = [{'kind': 'mse'         , 'weight': 1.0}]

LRPDict_theta = {}
lagStepsList = [30, 60]

kwargs = {'losses': Losses, 'optim': 'adam', 'metrics': ['mae'],
            'batch_size': 10, 'epochs': 50, 'validation_split': 0.1,
            'filename': 'model', 'dirname': os.path.abspath('')}

for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(X, y, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0'],
                                                       lagSteps = lagStepsList[i],
                                                       layers = Layers, **kwargs)

In [ ]:
lagStepsList = [90, 120]

for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(X, y, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0'],
                                                       lagSteps = lagStepsList[i],
                                                       layers = Layers, **kwargs)

In [ ]:
lagStepsList = [150, 180]

for i in range(len(lagStepsList)):

    print(f'Lag: {lagStepsList[i]} days, for Theta')
    LRPDict_theta[f'LRP{lagStepsList[i]}'] = quickSetup(X, y, numVars = 1, lrp_methods = ['lrp.alpha_1_beta_0'],
                                                       lagSteps = lagStepsList[i],
                                                       layers = Layers, **kwargs)

In [ ]:
plt.figure(figsize=(20,10))

field = np.mean(LRPDict_theta['LRP30']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,1])

P[1].set_title("LRP_a1b0 lag 30 days")

field = np.mean(LRPDict_theta['LRP60']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,2])

P[1].set_title("LRP_a1b0 lag 60 days")

field = np.mean(LRPDict_theta['LRP90']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,3])

P[1].set_title("LRP_a1b0 lag 90 days")

field = np.mean(LRPDict_theta['LRP120']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,4])

P[1].set_title("LRP_a1b0 lag 120 days")

field = np.mean(LRPDict_theta['LRP150']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,5])

P[1].set_title("LRP_a1b0 lag 150 days")

field = np.mean(LRPDict_theta['LRP180']['lrp.alpha_1_beta_0'], axis=(0,1))
field = field / np.nanmax(np.abs(field))

P = ecco.plot_proj_to_latlon_grid(ds.XC, ds.YC,
                              field,
                              plot_type = 'contourf',
                              show_colorbar=True, cmap='jet', cmin = 0, cmax = 1,
                              dx=2, dy=2, projection_type = 'robin',
                              less_output = True, subplot_grid = [2,3,6])

P[1].set_title("LRP_a1b0 lag 180 days")

In [ ]:
plt.plot(y_new[180:], 'black', label = 'True')
plt.plot(LRPDict_theta['LRP30']['cost_predict_NN'][150:], 'brown', label = 'lag 30 days')
plt.plot(LRPDict_theta['LRP60']['cost_predict_NN'][120:], 'r--', label = 'lag 60 days')
plt.plot(LRPDict_theta['LRP90']['cost_predict_NN'][90:], 'b-', label = 'lag 90 days')
plt.plot(LRPDict_theta['LRP120']['cost_predict_NN'][60:], 'orange', label = 'lag 120 days')
plt.plot(LRPDict_theta['LRP150']['cost_predict_NN'][30:], 'pink', label = 'lag 150 days')
plt.plot(LRPDict_theta['LRP180']['cost_predict_NN'], 'purple', label = 'lag 180 days')
plt.title("Predicted objective function")
plt.legend()

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow import optimizers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Input, Activation
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import layers
from sklearn.utils import shuffle
import tensorflow.keras.backend as kbackend
import tensorflow.keras.models
from keras.utils import np_utils
from keras.regularizers import l2

# Loading the CIFAR-10 datasets
from keras.datasets import cifar10

tf.compat.v1.disable_eager_execution()

In [ ]:
keras.backend.clear_session()

inputs = Input(shape=(90, 90, 13))
conv2d1 = Conv2D(filters=32,kernel_size=(8,8),input_shape=(90, 90, 13),activation='relu', kernel_regularizer=l2(0.001))
maxpool2d = MaxPooling2D(pool_size=(4,4))
dropout = Dropout(0.2)
conv2d2 = Conv2D(filters=64,kernel_size=(8,8),input_shape=(20, 20, 32),activation='relu', kernel_regularizer=l2(0.001))
flatten = Flatten()
dense1 = Dense(576, activation='relu', kernel_regularizer=l2(0.001))
dense2 = Dense(24, activation='relu')
dense3 = Dense(1, activation='linear')

x = conv2d1(inputs)
x = maxpool2d(x)
x = dropout(x)
x = conv2d2(x)
x = maxpool2d(x)
x = dropout(x)
x = flatten(x)
x = dense1(x)
x = dropout(x)
x = dense2(x)
x = dropout(x)
x = dense3(x)

model = keras.Model(inputs=inputs, outputs=x) 

mod_h5 = 'model.h5'

checkpoint = ModelCheckpoint(mod_h5, monitor='val_loss',
                             verbose=1,save_best_only=True)
        
callbacks = [checkpoint]
model.compile(optimizer='adam',
            loss='mse', metrics=['mae'])

fit = model.fit(X_2d[:-90], y[90:],
            batch_size=10,
            epochs=5, 
            shuffle=True,
            validation_split = 0.1, 
            callbacks=callbacks)  

In [ ]:
keras.backend.clear_session()

inputs = Input(shape=(90, 90, 13))
conv2d1 = Conv2D(filters=32,kernel_size=(8,8),input_shape=(90, 90, 13),activation='relu', kernel_regularizer=l2(0.001))
maxpool2d = MaxPooling2D(pool_size=(4,4))
dropout = Dropout(0.2)
conv2d2 = Conv2D(filters=64,kernel_size=(8,8),input_shape=(20, 20, 32),activation='relu', kernel_regularizer=l2(0.001))
flatten = Flatten()
dense1 = Dense(576, activation='relu', kernel_regularizer=l2(0.001))
dense2 = Dense(24, activation='relu')
dense3 = Dense(1, activation='linear')

x = conv2d1(inputs)
x = maxpool2d(x)
x = dropout(x)
x = conv2d2(x)
x = maxpool2d(x)
x = dropout(x)
x = flatten(x)
x = dense1(x)
x = dropout(x)
x = dense2(x)
x = dropout(x)
x = dense3(x)

best_model = keras.Model(inputs=inputs, outputs=x) 
best_model.load_weights(mod_h5)
best_model.compile(loss='mse', optimizer='adam',metrics=['mae'])

y_conv_pred = best_model.predict(X_2d)

In [ ]:
plt.plot(y[90:], 'b-')
plt.plot(y_conv_pred, 'r--')
plt.show()

In [ ]:
Xplain = XAIR(best_model, 'lrp.alpha_1_beta_0', 'classic', X_2d, normalizeDict, **kwargs)
a, _  = Xplain.quick_analyze()